In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1000.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  257.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4489 personas en los últimos 7 días 

Un positivo PCR cada 2817 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1000.0,151.0,282.0,510743.0,29.564771,55.213679,21.0
Huelva-Costa,528.0,92.0,148.0,288115.0,31.931694,51.368377,12.0
Huelva (capital),257.0,32.0,51.0,143663.0,22.274350,35.499746,8.0
Condado-Campiña,301.0,41.0,104.0,155057.0,26.441889,67.072109,4.0
Sierra de Huelva-Andévalo Central,153.0,14.0,20.0,67571.0,20.718947,29.598496,4.0
Aljaraque,56.0,10.0,13.0,21260.0,47.036689,61.147695,2.0
Lepe,67.0,8.0,16.0,27431.0,29.164084,58.328169,2.0
Palos de la Frontera,20.0,11.0,12.0,11289.0,97.439986,106.298166,2.0
Aroche,4.0,2.0,2.0,3073.0,65.082981,65.082981,1.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Cabezas Rubias,7.0,5.0,6.0,730.0,684.931507,821.917808,-1.0
Villalba del Alcor,37.0,7.0,24.0,3338.0,209.706411,718.993409,1.0
Escacena del Campo,15.0,3.0,11.0,2284.0,131.348511,481.611208,0.0
Zalamea la Real,19.0,5.0,6.0,3068.0,162.972621,195.567145,0.0
Isla Cristina,52.0,18.0,35.0,21264.0,84.650113,164.597442,0.0
Bollullos Par del Condado,35.0,12.0,23.0,14272.0,84.080717,161.154709,0.0
Cortegana,9.0,1.0,5.0,4666.0,21.431633,107.158165,NaN
Palos de la Frontera,20.0,11.0,12.0,11289.0,97.439986,106.298166,2.0
Santa Bárbara de Casa,7.0,1.0,1.0,1035.0,96.618357,96.618357,NaN
